In [1]:
import os
import json
import cv2
import pandas as pd
from tqdm import tqdm
import numpy as np


In [2]:
os.listdir()

['fixed_gan.ipynb',
 'simple_model.pth',
 '.DS_Store',
 'models.py',
 'measures_categories.csv',
 'score_category.ipynb',
 'icon_legend.json',
 'design_topics.csv',
 'measures_context.csv',
 '__pycache__',
 '10594-screenshot.jpg',
 'component_legend.json',
 'PretextTasks_SSL.ipynb',
 'README.md',
 'measures.csv',
 '.gitignore',
 '10594-hierarchy.json',
 'simple_model.pt',
 'MakeDataset.ipynb',
 'textButton_legend.json',
 'gflow_net.ipynb',
 'combined',
 '.git',
 '10594-wireframe.png',
 '10594-metadata.json',
 'model.ipynb',
 'model_clip.ipynb',
 'semantic_annotations']

1. Density - Anushka

In [3]:
import json

def calculate_area(bounds):
    return abs((bounds[2] - bounds[0]) * (bounds[3] - bounds[1]))

def do_rectangles_overlap(rect1, rect2):
    return not (rect1[2] <= rect2[0] or rect1[0] >= rect2[2] or rect1[3] <= rect2[1] or rect1[1] >= rect2[3])

def merge_overlapping_objects(data):
    merged_objects = []

    if 'children' in data:
        for parent in data['children']:
            non_overlapping_children = []

            if 'children' in parent:
                for child in parent['children']:
                    overlapping = False
                    for merged_obj in merged_objects:
                        if do_rectangles_overlap(child['bounds'], merged_obj['bounds']):
                            overlapping = True
                            break

                    if not overlapping:
                        non_overlapping_children.append(child)

                merged_objects.extend(non_overlapping_children)
     
    return merged_objects

def calculate_density_measure(total_area, frame_area):
    if frame_area == 0:
        return 0  # Handle division by zero scenario
    return 1 - 2 * abs(0.5 - total_area / frame_area)

def calculate_density(data):
    total_area = 0

    merged_objects = merge_overlapping_objects(data)

    for item in merged_objects:
        bounds = item.get('bounds')
        if bounds:
            area = calculate_area(bounds)
            total_area += abs(area)

    frame_bounds = data.get('bounds')
    if frame_bounds:
        frame_area = calculate_area(frame_bounds)
    else:
        return 0  # Handle case where frame bounds are not defined

    density_measure = calculate_density_measure(total_area, frame_area)

    return density_measure

2. Colour - Yash

In [4]:
def calculate_colorfulness(image):
    image = cv2.imread(image)
    # Convert the image to sRGB color space
    srgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Calculate the pixel cloud along directions (rg, yb)
    rg = srgb_image[:,:,0] - srgb_image[:,:,1]
    yb = (srgb_image[:,:,0] + srgb_image[:,:,1]) / 2 - srgb_image[:,:,2]

    # Calculate the standard deviation and mean value along directions (rg, yb)
    std_rg = np.std(rg)
    std_yb = np.std(yb)
    mean_rg = np.mean(rg)
    mean_yb = np.mean(yb)

    # Calculate ^M(3) colorfulness metric
    colorfulness = np.sqrt(std_rg**2 + std_yb**2) + 0.3 * np.sqrt(mean_rg**2 + mean_yb**2)
    #make colour between 0 and 1
    # colorfulness = colorfulness / 200

    return colorfulness

3. Proportion - Devesh

In [5]:
# 1640
# 39266
standard_proportions = {
        'sq': 1,        # Square
        'r2': 1/1.414,  # Square root of 2
        'gr': 1/1.618,  # Golden ratio
        'r3': 1/1.732,  # Square root of 3
        'ds': 1/2       # Double square
    }

# Recursive function to parse JSON and calculate PM_object
def parse_json_and_calculate_PMobject(component, standard_proportions):
    sum_min_diff = 0
    n = 0

    def extract_proportion(bounds):
        x1, y1, x2, y2 = bounds
        width = x2 - x1
        height = y2 - y1
        r = height / width if width != 0 else 0
        if width != 0 and r <= 1:
            return r
        elif width != 0 and r > 1:
            return 1/r
        else:
            return r

    # Recursively process components and children
    def process_component(component):
        nonlocal sum_min_diff, n
        bounds = component.get("bounds", [])
        if bounds:
            proportion = extract_proportion(bounds)
            min_diff = min(abs(proportion - sp) for sp in standard_proportions.values())
            sum_min_diff += (1 - min_diff / 0.5)
            n += 1
        for child in component.get("children", []):
            process_component(child)

    process_component(component)
    return abs(sum_min_diff / n) if n != 0 else 0

# Calculate the PM_object for the entire JSON data structure
# PM_object = parse_json_and_calculate_PMobject(json_data, standard_proportions)
# PM_object

def calculate_PMlayout(width_layout, height_layout):
    # Standard proportions as per the provided screenshot

    # Calculate the layout proportion
    r_layout = height_layout / width_layout if width_layout != 0 else 0
    p_layout = r_layout if r_layout <= 1 else 1 / r_layout

    # Find the minimum difference between p_layout and the standard proportions
    min_diff = min(abs(p_layout - sp) for sp in standard_proportions.values())

    # Calculate PM_layout according to the formula
    PM_layout = 1 - (min_diff / 0.5)

    return abs(PM_layout)

# calculate_PMlayout(json_data["bounds"][2],json_data["bounds"][3])

def calculate_PM(json_data):
    pm_object = parse_json_and_calculate_PMobject(json_data, standard_proportions)
    pm_layout = calculate_PMlayout(json_data["bounds"][2],json_data["bounds"][3])
    return (pm_object + pm_layout) / 2

In [6]:
# with open("semantic_annotations/39266.json") as f:
#     data = json.load(f)
    
# calculate_PM(data)

4. Symmetry - Mann

In [7]:
import os
import json

def determine_quadrant(element_center_x, element_center_y, center_x, center_y):
    if element_center_x < center_x and element_center_y < center_y:
        return 'UL'  # Upper-Left
    elif element_center_x >= center_x and element_center_y < center_y:
        return 'UR'  # Upper-Right
    elif element_center_x < center_x and element_center_y >= center_y:
        return 'LL'  # Lower-Left
    elif element_center_x >= center_x and element_center_y >= center_y:
        return 'LR'  # Lower-Right

def calculate_element_properties(bounds, screen_width, screen_height):
    if screen_width == 0 or screen_height == 0:
        print(f"Error: Screen width or height is zero. Width: {screen_width}, Height: {screen_height}, Bounds: {bounds}")
        return None, None, None, None  # Return None values if dimensions are zero

    width = (bounds[2] - bounds[0]) / screen_width
    height = (bounds[3] - bounds[1]) / screen_height
    center_x = (bounds[0] + width / 2) / screen_width
    center_y = (bounds[1] + height / 2) / screen_height
    return center_x, center_y, width, height

def extract_ui_elements(data, parent_bounds=None):
    elements = []
    bounds = data.get('bounds', parent_bounds)
    if 'children' not in data or not data['children']:
        return [{'class': data['class'], 'bounds': bounds}]
    for child in data['children']:
        elements.extend(extract_ui_elements(child, bounds))
    return elements

def calculate_symmetry(data):
    screen_bounds = data['bounds']
    screen_width = screen_bounds[2] - screen_bounds[0]
    screen_height = screen_bounds[3] - screen_bounds[1]

    if screen_width == 0 or screen_height == 0:
        # print(f"Error: Invalid screen dimensions. Bounds: {screen_bounds}")
        return 0  # Return 0 as the symmetry score if screen dimensions are zero

    ui_elements = extract_ui_elements(data)
    quadrants = {'UL': [], 'UR': [], 'LL': [], 'LR': []}
    for element in ui_elements:
        center_x, center_y, width, height = calculate_element_properties(element['bounds'], screen_width, screen_height)
        if center_x is None:  # Check if None values were returned
            continue  # Skip this element due to invalid dimensions
        quadrant = determine_quadrant(center_x, center_y, 0.5, 0.5)
        quadrants[quadrant].append({'center_x': center_x, 'center_y': center_y, 'width': width, 'height': height})

    symmetry_scores = {'SYM_vertical': [], 'SYM_horizontal': [], 'SYM_radial': []}
    pairs_vertical = [('UL', 'UR'), ('LL', 'LR')]
    pairs_horizontal = [('UL', 'LL'), ('UR', 'LR')]
    pairs_radial = [('UL', 'LR'), ('UR', 'LL')]

    for p1, p2 in pairs_vertical:
        for el1, el2 in zip(quadrants[p1], quadrants[p2]):
            symmetry_scores['SYM_vertical'].append(1 - abs(el1['center_x'] - el2['center_x']))

    for p1, p2 in pairs_horizontal:
        for el1, el2 in zip(quadrants[p1], quadrants[p2]):
            symmetry_scores['SYM_horizontal'].append(1 - abs(el1['center_y'] - el2['center_y']))

    for p1, p2 in pairs_radial:
        for el1, el2 in zip(quadrants[p1], quadrants[p2]):
            radial_diff_x = 1 - abs(el1['center_x'] - el2['center_x'])
            radial_diff_y = 1 - abs(el1['center_y'] - el2['center_y'])
            symmetry_scores['SYM_radial'].append((radial_diff_x + radial_diff_y) / 2)

    average_scores = {key: (sum(values) / len(values)) if values else 0 for key, values in symmetry_scores.items()}

    # Calculate the normalized final combined symmetry score
    final_symmetry_score = sum(average_scores.values()) / len(average_scores)
    return final_symmetry_score

5. Balance - Vedika

In [8]:
import json

def screen_bounds(json_data):
    bounds = json_data['bounds']
    left, top, right, bottom = bounds
    width = right - left
    height = bottom - top
    return width, height

def parse_annotations(json_data):
    objects = []
    
    def parse_children(children):
        for item in children:
            if 'bounds' in item:
                left, top, right, bottom = item['bounds']
                width = right - left
                height = bottom - top
                objects.append({
                    'left': left,
                    'top': top,
                    'right': right,
                    'bottom': bottom,
                    'width': width,
                    'height': height
                })
            if 'children' in item:
                parse_children(item['children'])

    parse_children(json_data['children'])
    return objects

def compute_balance_scores(objects, json_data):
    screen_width, screen_height = screen_bounds(json_data)
    left_area = right_area = top_area = bottom_area = 0
    left_distance = right_distance = top_distance = bottom_distance = 0
    
    if objects == []:
        return 0
    else:
        for obj in objects:
            if (obj['left'] + obj['right']) / 2 < screen_width / 2:
                left_area += obj['width'] * obj['height']
                left_distance += abs((obj['left'] + obj['width']) / 2 - screen_width)
            else:
                right_area += obj['width'] * obj['height']
                right_distance += abs((obj['left'] + obj['width']) / 2 - screen_width)
            
            if (obj['top'] + obj['bottom']) / 2 < screen_height / 2:
                top_area += obj['width'] * obj['height']
                top_distance += abs((obj['top'] + obj['height']) / 2 - screen_height)
            else:
                bottom_area += obj['width'] * obj['height']
                bottom_distance += abs((obj['top'] + obj['height']) / 2 - screen_height)
      
        left_weight = left_area / max(left_area, right_area) if max(left_area, right_area) != 0 else 0
        right_weight = right_area / max(left_area, right_area) if max(left_area, right_area) != 0 else 0
        vertical_balance = abs(left_weight - right_weight)

        top_weight = top_area / max(top_area, bottom_area) if max(top_area, bottom_area) != 0 else 0
        bottom_weight = bottom_area / max(top_area, bottom_area) if max(top_area, bottom_area) != 0 else 0
        horizontal_balance = abs(top_weight - bottom_weight)
      
        balance_measure = abs((vertical_balance + horizontal_balance) / 2)
    
    return balance_measure

def balance_score(json_data):
    objects = parse_annotations(json_data)
    balance_score = compute_balance_scores(objects, json_data)
    # print("Balance Score:", balance_score)
    return balance_score

**Calculate Everything**

In [9]:
# import zipfile
# import os

# # Assuming we have a path to the zip file and a target directory
# zip_file_path = './ui_layout_vectors.zip'  # Replace with your zip file path
# target_directory = './'  # Replace with your target directory

# # Create target directory if it does not exist
# if not os.path.exists(target_directory):
#     os.makedirs(target_directory)

# # Extract the zip file
# with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
#     zip_ref.extractall(target_directory)

# # The code above assumes the paths are known and correctly provided.
# # If you have the zip file already in your environment, you can adjust the paths accordingly.


In [10]:
topics = pd.read_csv("design_topics.csv")
mapping_ids = []
for index,i in topics.iterrows():
    mapping_ids.append(i.values)

In [11]:
output_csv = './measures.csv'
json_folder = './semantic_annotations'
image_folder = './combined'

page_names = []
balance_measures = []
colour_measures = []
symmetry_measures = []
proportion_measures = []
density_measures = []

progress_bar = tqdm(total=len(os.listdir(json_folder)), desc="Processing JSON files")

for filename in os.listdir(json_folder):
    if filename.endswith('.json'):
        progress_bar.update(1)  # Update progress bar
        page_number = filename.split('.')[0]
        page_names.append(page_number)

        with open(os.path.join(json_folder, filename), 'r', encoding='utf-8') as f:
            data = json.load(f)

        balance = balance_score(data)
        balance_measures.append(balance)

        proportion = calculate_PM(data)
        proportion_measures.append(proportion)

        symmetry = calculate_symmetry(data)
        symmetry_measures.append(symmetry)

        density = calculate_density(data)
        density_measures.append(density)

        # Load corresponding image for colour calculation
        image_filename = page_number + '.jpg'
        image_path = os.path.join(image_folder, image_filename)
        # print(image_path)
        if os.path.exists(image_path):
            colour = calculate_colorfulness(image_path)
            colour_measures.append(colour)
            # print(f"Color value for {page_number} : {colour}")
        else:
            # print("no file exists")
            colour_measures.append(None)  # Handle case where image is not found
        # print("------")

        # Log progress every 1000 pages
        if len(page_names) % 2000 == 0:
            print(f"Processed {len(page_names)} pages")

progress_bar.close()

# Create a DataFrame to store the results
results_df = pd.DataFrame({
    'Page': page_names,
    'Balance': balance_measures,
    'Colour': colour_measures,
    'Symmetry': symmetry_measures,
    'Proportion': proportion_measures,
    'Density': density_measures
})

# Calculate final score (average of all measures)
results_df['Colour'] = results_df['Colour'] / 200 # to scale between 0-1
results_df['Final Score'] = results_df[['Balance', 'Colour', 'Symmetry', 'Proportion', 'Density']].mean(axis=1)
# results_df['Final Score'] = results_df[['Balance', 'Symmetry', 'Proportion', 'Density']].mean(axis=1)

# Filtering outliers (dropping 1 out of 300 entries, negligible)
results_df = results_df[results_df['Final Score'] <= 1] 

# Save results to CSV
results_df.to_csv(output_csv, index=False)

print("Results saved to", output_csv)

Processing JSON files:   2%|▏         | 2008/132524 [00:25<27:05, 80.27it/s]

Processed 2000 pages


Processing JSON files:   3%|▎         | 4011/132524 [00:53<29:23, 72.87it/s]

Processed 4000 pages


Processing JSON files:   5%|▍         | 6014/132524 [01:20<28:40, 73.51it/s]

...

Processed 64000 pages


Processing JSON files:  50%|████▉     | 66014/132524 [14:53<15:13, 72.79it/s]

Processed 66000 pages


Processing JSON files:  50%|████▉     | 66261/132524 [14:56<14:56, 73.88it/s]


Results saved to ./measures.csv


In [12]:
output_csv = './measures_context.csv'
json_folder = './semantic_annotations'
image_folder = './combined'

page_names = []
balance_measures = []
colour_measures = []
symmetry_measures = []
proportion_measures = []
density_measures = []
descriptions = []

progress_bar = tqdm(total=len(os.listdir(json_folder)), desc="Processing JSON files")

for c,i in enumerate(mapping_ids):
    filename = f"{i[0]}.json"
    if filename.endswith('.json'):
        progress_bar.update(1)  # Update progress bar

        if os.path.isfile(os.path.join(json_folder, filename)):
            with open(os.path.join(json_folder, filename), 'r', encoding='utf-8') as f:
                data = json.load(f)
        else:
            continue
        
        page_number = filename.split('.')[0]
        page_names.append(page_number)

        balance = balance_score(data)
        balance_measures.append(balance)

        proportion = calculate_PM(data)
        proportion_measures.append(proportion)

        symmetry = calculate_symmetry(data)
        symmetry_measures.append(symmetry)

        density = calculate_density(data)
        density_measures.append(density)
        
        descriptions.append(i[1])

        # Load corresponding image for colour calculation
        image_filename = page_number + '.jpg'
        image_path = os.path.join(image_folder, image_filename)
        # print(image_path)
        if os.path.exists(image_path):
            colour = calculate_colorfulness(image_path)
            colour_measures.append(colour)
            # print(f"Color value for {page_number} : {colour}")
        else:
            # print("no file exists")
            colour_measures.append(None)  # Handle case where image is not found
        # print("------")

        # Log progress every 1000 pages
        if len(page_names) % 2000 == 0:
            print(f"Processed {len(page_names)} pages")

progress_bar.close()

# Create a DataFrame to store the results
results_df = pd.DataFrame({
    'Page': page_names,
    'Balance': balance_measures,
    'Colour': colour_measures,
    'Symmetry': symmetry_measures,
    'Proportion': proportion_measures,
    'Density': density_measures,
    'Description': descriptions
})

# Calculate final score (average of all measures)
results_df['Colour'] = results_df['Colour'] / 200 # to scale between 0-1
results_df['Final Score'] = results_df[['Balance', 'Colour', 'Symmetry', 'Proportion', 'Density']].mean(axis=1)
# results_df['Final Score'] = results_df[['Balance', 'Symmetry', 'Proportion', 'Density']].mean(axis=1)

# Filtering outliers (dropping 1 out of 300 entries, negligible)
results_df = results_df[results_df['Final Score'] <= 1] 

# Save results to CSV
results_df.to_csv(output_csv, index=False)

print("Results saved to", output_csv)

Processing JSON files:   1%|          | 1460/132524 [00:17<26:45, 81.66it/s]

Results saved to ./measures_context.csv


In [13]:
df = pd.read_csv("measures.csv")
df.describe()

,Page,Balance,Colour,Symmetry,Proportion,Density,Final Score
count,66249.000000,66249.000000,66249.000000,66249.000000,66249.000000,66249.000000,66249.000000
mean,36072.580703,0.695085,0.598781,0.270729,0.810199,0.167850,0.508529
std,20904.422841,0.243705,0.175356,0.168905,0.084053,23.422624,4.687986
min,0.000000,0.000000,0.000000,-4.063639,0.000000,-6019.250000,-1204.162904
25%,17856.000000,0.519113,0.487014,0.130729,0.757232,0.000000,0.473365
50%,36178.000000,0.734394,0.599987,0.294345,0.808285,0.142700,0.525395
75%,54241.000000,0.908731,0.730558,0.401270,0.859998,0.501606,0.582455
max,72218.000000,1.732173,1.262280,0.892908,2.327174,1.000000,0.865920


In [14]:
df[df["Balance"] > 1]

,Page,Balance,Colour,Symmetry,Proportion,Density,Final Score
235,14518,1.108263,0.528254,0.627917,0.820340,0.790295,0.775014
3087,4748,1.330486,0.688026,-0.015252,0.610342,0.106337,0.543988
10391,49190,1.162551,0.637811,0.295081,0.771291,-0.481969,0.476953
11365,34818,1.191943,0.390435,0.195338,0.550000,0.157432,0.497029
24507,14070,1.116827,0.566883,0.374526,0.730774,0.067961,0.571394
28526,56445,1.293240,0.700323,0.521112,0.750429,0.000000,0.653021
29292,53775,1.272726,0.860029,0.278758,0.573512,0.351685,0.667342
29752,8849,1.732173,0.536671,0.370936,0.590874,0.902919,0.826715
36859,8165,1.692374,0.555232,-0.006579,0.498826,0.000000,0.547970
42832,52015,1.170661,0.376859,0.339959,0.590300,0.563192,0.608194


In [15]:
df[df["Proportion"] > 1]

,Page,Balance,Colour,Symmetry,Proportion,Density,Final Score
6486,1511,0.564834,0.775499,0.435664,1.113131,0.146321,0.607090
8186,38816,0.608375,0.655269,0.464537,1.723629,0.877790,0.865920
8418,38817,0.440054,0.670884,0.433090,2.321343,0.339396,0.840953
12979,40554,0.429681,0.619110,0.417983,1.623590,0.295102,0.677093
30057,52349,0.565720,0.601457,0.403326,2.140121,0.183826,0.778890
30866,60900,0.565720,0.602002,0.403326,2.140121,0.183826,0.778999
39574,20686,0.859946,0.821130,0.392135,1.212113,0.388206,0.734706
49503,38813,0.765944,0.671781,0.415454,1.858556,0.427490,0.827845
52701,38822,0.439952,0.630182,0.433090,2.327174,0.342037,0.834487
56149,2981,1.018943,0.839108,0.151610,1.231169,0.776430,0.803452


In [16]:
df[df["Colour"] > 1].count()

Page           10
Balance        10
Colour         10
Symmetry       10
Proportion     10
Density        10
Final Score    10
dtype: int64

In [17]:
df[ df["Density"] == 0 ].count()

Page           16993
Balance        16993
Colour         16993
Symmetry       16993
Proportion     16993
Density        16993
Final Score    16993
dtype: int64

In [18]:
df[ df["Symmetry"] == 0 ].count()

Page           6895
Balance        6895
Colour         6895
Symmetry       6895
Proportion     6895
Density        6895
Final Score    6895
dtype: int64

In [19]:
df[ df["Proportion"] == 0 ].count()

Page           66
Balance        66
Colour         66
Symmetry       66
Proportion     66
Density        66
Final Score    66
dtype: int64

In [20]:
df[ df["Balance"] == 0 ].count()

Page           724
Balance        724
Colour         724
Symmetry       724
Proportion     724
Density        724
Final Score    724
dtype: int64